In [1]:
!pip install pyspark

     |████████████████████████████████| 193.9MB 1.2MB/s 
     |████████████████████████████████| 204kB 43.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/dc/0e/02/e9fdf0bf3ad20284175307d4ab31afcf967604f25f3b4f1d96
Successfully built pyspark


In [2]:
from google.colab import drive
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.mllib.linalg import Vectors
from pyspark.sql.functions import udf, array, avg, col, size, struct, lag, window, countDistinct, sum
from pyspark.sql.types import DoubleType, IntegerType, StringType, ArrayType, LongType, FloatType
from pyspark.sql.window import Window
import re

drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
spark = SparkSession.builder.getOrCreate()
directory = '/content/gdrive/My Drive/CS 205 Project/records/'

In [0]:
from pyspark.sql.functions import udf, array, avg, col, size, struct, lag, window, countDistinct, sum

# Load data

In [0]:
df = spark.read.csv(directory + 'first_llorente_video.csv', header='true', multiLine='true')

In [167]:
df_test = spark.read.json(directory + '1-min')

DataFrame[bboxes: array<double>, scores: array<double>]

In [168]:
df_test.show()

+--------------------+--------------------+
|              bboxes|              scores|
+--------------------+--------------------+
|[0.59336024522781...|[0.93980032205581...|
|[0.62502855062484...|[0.89926099777221...|
|[0.72911220788955...|[0.98777532577514...|
|[0.78701549768447...|[0.97126829624176...|
|[0.48519986867904...|[0.93738293647766...|
|[0.67814970016479...|[0.97286134958267...|
|[0.57940828800201...|[0.98793494701385...|
|[0.71229845285415...|[0.98705601692199...|
|[0.34067243337631...|[0.77425789833068...|
|[0.78984218835830...|[0.97743964195251...|
|[0.39440840482711...|[0.92999798059463...|
|[0.60313117504119...|[0.93863993883132...|
|[0.82778960466384...|[0.91649687290191...|
|[0.51006656885147...|[0.97323024272918...|
|[0.56186217069625...|[0.89469927549362...|
|[0.67285186052322...|[0.97309362888336...|
|[0.35979881882667...|[0.84619909524917...|
|[0.39222764968872...|[0.71783727407455...|
|[0.47849225997924...|[0.92085105180740...|
|[0.51487535238265...|[0.8125121

In [169]:
df_test.withColumn('bboxes test', size('bboxes')).withColumn('scores test', size('scores')).show()

+--------------------+--------------------+-----------+-----------+
|              bboxes|              scores|bboxes test|scores test|
+--------------------+--------------------+-----------+-----------+
|[0.59336024522781...|[0.93980032205581...|         36|          9|
|[0.62502855062484...|[0.89926099777221...|         44|         11|
|[0.72911220788955...|[0.98777532577514...|         36|          9|
|[0.78701549768447...|[0.97126829624176...|         36|          9|
|[0.48519986867904...|[0.93738293647766...|         36|          9|
|[0.67814970016479...|[0.97286134958267...|         32|          8|
|[0.57940828800201...|[0.98793494701385...|         32|          8|
|[0.71229845285415...|[0.98705601692199...|         24|          6|
|[0.34067243337631...|[0.77425789833068...|         28|          7|
|[0.78984218835830...|[0.97743964195251...|         24|          6|
|[0.39440840482711...|[0.92999798059463...|         28|          7|
|[0.60313117504119...|[0.93863993883132...|     

# Compute Velocity and Group Size

In [0]:
def compute_velocities(cols, fps=2, threshold=0.5):
    # Assume frame_1 = [x_i, y_i, x_{i+1}, y_{i+1}, ...]
    f_1 = iter(cols[0])
    f_2 = iter(cols[1])
    frame_1 = list(zip(f_1, f_1))
    frame_2 = list(zip(f_2, f_2))
    
    val_1 = {k: v for k, v in enumerate(frame_1)}
    val_2 = {k: v for k, v in enumerate(frame_2)}

    # Compute pairwise distances
    distances = {}
    for i in range(len(frame_1)):
        for j in range(len(frame_2)):
            # Define distance between two ids (from separate frames) as
            # the max difference between x coord and y coord
#             distances[i, j] = max(
#                 abs(val_1[i][0] - val_2[j][0]), abs(val_1[i][1] - val_2[j][1]))
            # Alternatively, use Euclidean distance
            distances[i, j] = np.sqrt(
                (val_1[i][0] - val_2[j][0]) ** 2 + (val_1[i][1] - val_2[j][1]) ** 2)

    # Assigned ids from frame 1 (reference frame), {id_i: vel_i}
    velocities = dict()

    # Assigned ids from frame 2 (with values as match in frame 1)
    targets = dict()
    num_assigned = 0
    num_ids = min(len(frame_1), len(frame_2))

    # Sort distances by key: (id in frame 1, id in frame 2)
    pairs = sorted(distances.items(), key=lambda v:v[1])
    for p, dist in pairs:
        # Stop assigning ids when the distance exceeds a user-defined threshold
        # i.e. this covers the case when a person leaves one end of the image
        # and another person enters at the opposite side. We should not match
        # these ids to each other.
        if dist > threshold:
            break

        # Found closest ids between frames
        if p[0] not in velocities and p[1] not in targets and num_assigned < num_ids:
            num_assigned += 1
            # Velocity (distance units per second)
            velocities[p[0]] = dist * fps
            targets[p[1]] = p[0]

            
    return [float(v) for v in velocities.values()]

  
def dfs_all(graph):
    def dfs(node, graph):
        stack = [node]
        cc = [node]
        while stack:
            u = stack.pop()
            for v in graph[u]:
                if not visited[v]:
                    visited[v] = True
                    cc.append(v)
                    stack.append(v)
        return cc

    ccs = []
    visited = [False for _ in range(len(graph))]
    for i in range(len(graph)):
        if not visited[i]:
            visited[i] = True
            cc = dfs(i, graph)
            ccs.append(cc)
    return list(map(len, ccs))

  
def compute_groups(positions, threshold=0.1):
    p_1 = iter(positions)
    positions = list(zip(p_1, p_1))
                   
    # Compute pairwise distances
    graph = {i: set() for i in range(len(positions))}
    for i in range(len(positions)):
        for j in range(i, len(positions)):
            # Euclidean distance between two people
            dist = np.sqrt(
                (positions[i][0] - positions[j][0]) ** 2 + (positions[i][1] - positions[j][1]) ** 2)
            # Add edge to graph 
            if dist < threshold:
                graph[i].add(j)
                graph[j].add(i)
    lengths = dfs_all(graph)           
    return lengths

# UDFs

In [0]:
def parse_arr(arr_str):
    elements = re.split(r'\s*', arr_str.replace('[', '').replace(']', '').strip())
    result = [float(x.strip()) for x in elements]
    return result

def drop_zeros(columns):
    return columns[0][:columns[-1]]

def drop_zeros_bbox(columns):
    return columns[0][:columns[-1]*4]
  
def find_people_indices(classes):
    return [i for i, val in enumerate(classes) if val == 1.0]
  
def filter_people(columns):
    return list(np.array(columns[0])[[0, 1]])
#     return list(np.array(columns[0])[columns[-1]])
  
def filter_people_bbox(columns):
    return [columns[0][4*i:i+4] for i in columns[-1]]  
  
def find_zeros(scores):
    return scores.index(0.)

def count(column):
    return len(column)
  
def sum_vals(column):
    return sum(column)
  
def avg_vals(columns):
    return columns[1] / columns[0]
  
def center_pos(values):
    res = []
    for i in range(int(len(values)/4)):
        y1, x1, y2, x2 = values[4*i:4*i+4]
        x_mean, y_mean = round(float(x1+x2)/2, 3), round(float(y1+y2)/2, 3)
        res.extend([x_mean, y_mean])
    return res

In [0]:
parse_udf = udf(parse_arr, ArrayType(DoubleType()))
filter_udf = udf(find_zeros, IntegerType())
people_udf = udf(find_people_indices, ArrayType(IntegerType()))
count_udf = udf(count, IntegerType())
sum_udf = udf(sum_vals, IntegerType())
sum_floats_udf = udf(sum_vals, DoubleType())
avg_udf = udf(lambda arr: avg_vals(arr), DoubleType())
center_udf = udf(center_pos, ArrayType(FloatType()))
velocity_udf = udf(lambda arr: compute_velocities(arr), ArrayType(DoubleType()))
group_udf = udf(compute_groups, ArrayType(IntegerType()))

# Filter out null predictions (score of 0)
filter_cols_udf = udf(lambda arr: drop_zeros(arr), ArrayType(DoubleType()))
filter_cols_ppl_udf = udf(lambda arr: drop_zeros(arr), ArrayType(IntegerType()))
filter_cols_bbox_udf = udf(lambda arr: drop_zeros_bbox(arr), ArrayType(DoubleType()))

# Store results from valid people
# filter_people_udf = udf(lambda arr: filter_people(arr), ArrayType(DoubleType()))
# filter_people_bbox_udf = udf(lambda arr: filter_people_bbox(arr), ArrayType(DoubleType()))

# Window size for pairwise shifting

In [0]:
w = Window().partitionBy().orderBy(col("frame"))

# Modify and Create Columns

In [161]:
df = df.withColumn('timestamp', df['timestamp'].cast('timestamp'))
df = df.withColumn('bboxes', parse_udf('bboxes'))
df = df.withColumn('scores', parse_udf('scores'))
df = df.withColumn('classes', parse_udf('classes'))
df = df.withColumn('num_people', filter_udf('scores'))
df = df.withColumn('people_indices', people_udf('classes'))
df = df.withColumn('scores', filter_cols_udf(struct('scores', 'num_people')))
df = df.withColumn('classes', filter_cols_udf(struct('classes', 'num_people')))
df = df.withColumn('bboxes', filter_cols_bbox_udf(struct('bboxes', 'num_people')))
df = df.withColumn('people_indices', filter_cols_ppl_udf(struct('people_indices', 'num_people')))
df = df.withColumn('frame', df['frame'].cast(LongType()))
df = df.withColumn('centers', center_udf('bboxes'))
df = df.withColumn('group_sizes', group_udf('centers'))
df = df.withColumn('num_groups', count_udf('group_sizes'))
df = df.withColumn('next_frame_centers', lag("centers", -1).over(w)).na.drop()
df = df.withColumn('velocities', velocity_udf(struct('centers', 'next_frame_centers')))
df.show()

+---+--------------------+-----+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+
|_c0|           timestamp|frame|              bboxes|              scores|             classes|num_people|      people_indices|             centers|         group_sizes|num_groups|  next_frame_centers|          velocities|
+---+--------------------+-----+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+
|  0|2019-04-06 01:54:...|    0|[0.5924571, 0.459...|[0.9862345, 0.984...|[1.0, 8.0, 1.0, 1...|        13|[0, 2, 3, 4, 5, 6...|[0.469, 0.639, 0....|[1, 1, 1, 7, 1, 1...|         7|[0.734, 0.685, 0....|[0.00282839070986...|
|  1|2019-04-06 01:54:...|    1|[0.55876905, 0.61...|[0.97783583, 0.97...|[8.0, 1.0, 1.0, 1...|        12|[1

In [0]:
df = df.withColumn('num_velocities', count_udf('velocities')).withColumn('sum_velocities', sum_floats_udf('velocities'))

In [171]:
df.show()

+---+--------------------+-----+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------+--------------------+
|_c0|           timestamp|frame|              bboxes|              scores|             classes|num_people|      people_indices|             centers|         group_sizes|num_groups|  next_frame_centers|          velocities|num_velocities|      sum_velocities|
+---+--------------------+-----+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------+--------------------+
|  0|2019-04-06 01:54:...|    0|[0.5924571, 0.459...|[0.9862345, 0.984...|[1.0, 8.0, 1.0, 1...|        13|[0, 2, 3, 4, 5, 6...|[0.469, 0.639, 0....|[1, 1, 1, 7, 1, 1...|         7|[0.734, 0.685, 0....|[0.00282839070986...| 

# Aggregate each 5 minute window to compute:
- average group size
- average velocity
- average number of people detected

In [0]:
agg_df = df.groupBy(window("timestamp", "5 minutes")).agg(avg('num_people'),sum('num_people'),sum('num_groups'),sum('sum_velocities'),sum('num_velocities'))

In [0]:
agg_df = agg_df.withColumn('avg_group_size', avg_udf(struct('sum(num_groups)', 'sum(num_people)'))).withColumn('avg_velocity', avg_udf(struct('sum(num_velocities)', 'sum(sum_velocities)')))

In [184]:
agg_df.show()

+--------------------+-----------------+---------------+---------------+-------------------+-------------------+------------------+-------------------+
|              window|  avg(num_people)|sum(num_people)|sum(num_groups)|sum(sum_velocities)|sum(num_velocities)|    avg_group_size|       avg_velocity|
+--------------------+-----------------+---------------+---------------+-------------------+-------------------+------------------+-------------------+
|[2019-04-06 01:50...|8.812338799254476|         468085|         287439|   21477.1833018067|             430217|1.6284672574007006|0.04992174484459401|
+--------------------+-----------------+---------------+---------------+-------------------+-------------------+------------------+-------------------+



In [177]:
agg_df.show()

+--------------------+---------------+---------------+-------------------+-------------------+------------------+-------------------+
|              window|sum(num_people)|sum(num_groups)|sum(sum_velocities)|sum(num_velocities)|    avg_group_size|       avg_velocity|
+--------------------+---------------+---------------+-------------------+-------------------+------------------+-------------------+
|[2019-04-06 01:50...|         468085|         287439|   21477.1833018067|             430217|1.6284672574007006|0.04992174484459401|
+--------------------+---------------+---------------+-------------------+-------------------+------------------+-------------------+



# Extra code

In [0]:
w = Window().partitionBy().orderBy(col("frame"))
fps = 2

# Units of minutes
window_minutes = 5
window_seconds = window_minutes * 60

In [70]:
df.withColumn('shifted', lag("group_sizes", window_seconds * fps).over(w)).na.drop().limit(10).toPandas()

,_c0,timestamp,frame,bboxes,scores,classes,num_valid,people_indices,group_sizes,shifted
0,600,2019-04-06 01:54:54.350262,600,"[0.5564988, 0.6051574, 0.83079064, 0.8617946, ...","[0.9569086, 0.9345962, 0.9022663, 0.5717952, 0...","[8.0, 1.0, 1.0, 18.0, 49.0, 62.0, 8.0, 10.0, 1...",8,"[1, 2, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[100, 3, 4, 1]","[100, 3, 4, 2]"
1,601,2019-04-06 01:54:54.350262,601,"[0.55633837, 0.6051136, 0.8308019, 0.8618444, ...","[0.9572116, 0.932251, 0.46985915, 0.39988732, ...","[8.0, 1.0, 8.0, 18.0, 62.0, 49.0, 10.0, 1.0, 1...",7,"[1, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[100, 3, 4, 1]","[100, 3, 4, 3]"
2,602,2019-04-06 01:54:54.350262,602,"[0.55513835, 0.60421175, 0.8289719, 0.8616411,...","[0.9585405, 0.64866894, 0.54278594, 0.45582318...","[8.0, 1.0, 1.0, 8.0, 62.0, 10.0, 1.0, 1.0, 1.0...",6,"[1, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","[100, 3, 4, 0]","[100, 3, 4, 2]"
3,603,2019-04-06 01:54:54.350262,603,"[0.55514705, 0.60438603, 0.8283029, 0.861342, ...","[0.95708096, 0.6157852, 0.5999863, 0.45717746,...","[8.0, 1.0, 44.0, 44.0, 8.0, 62.0, 10.0, 1.0, 1...",7,"[1, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[100, 3, 4, 2]","[100, 3, 4, 2]"
4,604,2019-04-06 01:54:54.350262,604,"[0.55518764, 0.6043623, 0.8274987, 0.86266273,...","[0.95363075, 0.6808314, 0.47482514, 0.41447762...","[8.0, 1.0, 44.0, 8.0, 62.0, 49.0, 10.0, 1.0, 1...",7,"[1, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[100, 3, 4, 0]","[100, 3, 4, 0]"
5,605,2019-04-06 01:54:54.350262,605,"[0.55474883, 0.6041525, 0.82736427, 0.8620648,...","[0.95424896, 0.92147774, 0.7168982, 0.6419136,...","[8.0, 1.0, 1.0, 1.0, 49.0, 8.0, 62.0, 10.0, 1....",8,"[1, 2, 3, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[100, 3, 4, 3]","[100, 3, 4, 2]"
6,606,2019-04-06 01:54:54.350262,606,"[0.5549574, 0.6041098, 0.8275656, 0.8619949, 0...","[0.95343703, 0.88541216, 0.7709682, 0.70414233...","[8.0, 1.0, 1.0, 1.0, 49.0, 8.0, 62.0, 1.0, 1.0...",12,"[1, 2, 3, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18...","[100, 3, 4, 0]","[100, 3, 4, 2]"
7,607,2019-04-06 01:54:54.350262,607,"[0.55539465, 0.60399354, 0.8273619, 0.86194587...","[0.95520896, 0.81845707, 0.76817584, 0.7071279...","[8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 62.0, 8.0, 49.0...",10,"[1, 2, 3, 4, 5, 10, 11, 12, 13, 14, 15, 16, 17...","[100, 3, 4, 2]","[100, 3, 4, 0]"
8,608,2019-04-06 01:54:54.350262,608,"[0.5522283, 0.61008024, 0.81290096, 0.8692812,...","[0.9290856, 0.59318215, 0.41274893, 0.390738, ...","[8.0, 14.0, 7.0, 1.0, 62.0, 44.0, 1.0, 1.0, 1....",6,"[3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[100, 3, 4, 2]","[100, 3, 4, 0]"
9,609,2019-04-06 01:54:54.350262,609,"[0.5529724, 0.610268, 0.81309533, 0.8688983, 0...","[0.9357266, 0.6040579, 0.40961322, 0.3446233, ...","[8.0, 14.0, 1.0, 7.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",4,"[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[100, 3, 4, 0]","[100, 3, 4, 0]"


In [67]:
df.limit(10).toPandas()

,_c0,timestamp,frame,bboxes,scores,classes,num_valid,people_indices,group_sizes
0,0,2019-04-06 01:54:54.350262,0,"[0.5924571, 0.45952806, 0.686525, 0.47779706, ...","[0.9862345, 0.9846124, 0.96301347, 0.85467225,...","[1.0, 8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",13,"[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[100, 3, 4, 2]"
1,1,2019-04-06 01:54:54.350262,1,"[0.55876905, 0.6130008, 0.81024545, 0.8550498,...","[0.97783583, 0.97685575, 0.7071147, 0.66160876...","[8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",12,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15...","[100, 3, 4, 3]"
2,2,2019-04-06 01:54:54.350262,2,"[0.55879766, 0.612742, 0.80924284, 0.8545442, ...","[0.97587866, 0.97196656, 0.8321477, 0.82237643...","[8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 16.0, 1.0, 1.0,...",11,"[1, 2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 14, 15, 1...","[100, 3, 4, 2]"
3,3,2019-04-06 01:54:54.350262,3,"[0.585739, 0.4640477, 0.683594, 0.48238125, 0....","[0.98206156, 0.97774976, 0.93095094, 0.8632616...","[1.0, 8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",17,"[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[100, 3, 4, 2]"
4,4,2019-04-06 01:54:54.350262,4,"[0.5861195, 0.46531188, 0.68410474, 0.48530918...","[0.9818289, 0.98028356, 0.72216564, 0.7162687,...","[1.0, 8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",11,"[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[100, 3, 4, 0]"
5,5,2019-04-06 01:54:54.350262,5,"[0.5595339, 0.6155165, 0.8104876, 0.8549662, 0...","[0.98193616, 0.96152925, 0.7210958, 0.7036052,...","[8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",13,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[100, 3, 4, 2]"
6,6,2019-04-06 01:54:54.350262,6,"[0.56109095, 0.6154148, 0.8113807, 0.8556289, ...","[0.98374695, 0.95496714, 0.9484551, 0.85788023...","[8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",13,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[100, 3, 4, 2]"
7,7,2019-04-06 01:54:54.350262,7,"[0.843129, 0.5024977, 0.96392983, 0.5332773, 0...","[0.9861422, 0.98345333, 0.98037064, 0.88236, 0...","[1.0, 8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",17,"[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[100, 3, 4, 0]"
8,8,2019-04-06 01:54:54.350262,8,"[0.84246117, 0.49912694, 0.95856214, 0.530545,...","[0.9946509, 0.98235714, 0.98181605, 0.8806909,...","[1.0, 8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",14,"[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[100, 3, 4, 0]"
9,9,2019-04-06 01:54:54.350262,9,"[0.8466456, 0.4944173, 0.9589074, 0.52855814, ...","[0.993614, 0.98047435, 0.97441375, 0.81668174,...","[1.0, 8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",17,"[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[100, 3, 4, 0]"


In [83]:
df.withColumn('num_groups', count_udf('group_sizes')).show()

+---+--------------------+-----+--------------------+--------------------+--------------------+---------+--------------------+--------------+----------+
|_c0|           timestamp|frame|              bboxes|              scores|             classes|num_valid|      people_indices|   group_sizes|num_groups|
+---+--------------------+-----+--------------------+--------------------+--------------------+---------+--------------------+--------------+----------+
|  0|2019-04-06 01:54:...|    0|[0.5924571, 0.459...|[0.9862345, 0.984...|[1.0, 8.0, 1.0, 1...|       13|[0, 2, 3, 4, 5, 6...|[100, 3, 4, 2]|         4|
|  1|2019-04-06 01:54:...|    1|[0.55876905, 0.61...|[0.97783583, 0.97...|[8.0, 1.0, 1.0, 1...|       12|[1, 2, 3, 4, 5, 6...|[100, 3, 4, 2]|         4|
|  2|2019-04-06 01:54:...|    2|[0.55879766, 0.61...|[0.97587866, 0.97...|[8.0, 1.0, 1.0, 1...|       11|[1, 2, 3, 4, 5, 7...|[100, 3, 4, 0]|         4|
|  3|2019-04-06 01:54:...|    3|[0.585739, 0.4640...|[0.98206156, 0.97...|[1.0, 8.

In [0]:
# df.groupBy("group_sizes", window("timestamp", "2 seconds")).agg(countDistinct("classes"))

In [0]:
df = spark.read.csv(directory + 'time_data.csv', header='true', multiLine='true')

In [5]:
df.show()

+---------+--------------------+------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+---------------------------------

In [0]:
w = Window().partitionBy().orderBy(col("Timestamp"))

df.withColumn('shifted', lag("data").over(w)).na.drop().show()